In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import wandb

# wandb.login()
os.environ['WANDB_API_KEY'] = ''

wandb.init(project='trace-example')

wandb: Currently logged in as: whatwant to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


## 1. OpenAI API 로깅

In [ ]:
import datetime
from openai import OpenAI
from wandb.sdk.data_types.trace_tree import Trace

os.environ['OPENAI_API_KEY'] = ''

client = OpenAI()

query = "대한민국의 수도는 어디야?"
temperature = 0.2
model_name = "gpt-3.5-turbo"

system_message = "You are a helpful assistant."
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': query}
]

response = client.chat.completions.create(
    model=model_name, messages=messages, temperature=temperature
)

root_span = Trace(
    name='root_span',
    kind='llm',
    status_code='success',
    status_message=None,
    metadata={
        'temperature': temperature,
        'token_usage': dict(response.usage),
        'model_name': model_name
    },
    inputs={'system_prompt': system_message, 'query': query},
    outputs={'response': response.choices[0].message.content}
)

root_span.log(name='openai_trace')

## 2. LlamaIndex 로깅

In [ ]:
%%capture
! pip install datasets
! pip install llama_index
! pip install llama-index-callbacks-wandb

In [ ]:
from datasets import load_dataset
import llama_index
from llama_index.core import Document, VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.core import set_global_handler
from llama_index.core.settings import Settings

model_name = 'gpt-3.5-turbo'

llm = OpenAI(model=model_name, temperature=0)

set_global_handler('wandb', run_args={'project': 'llamaindex'})
wandb_callback = llama_index.core.global_handler

Settings.llm = llm

dataset = load_dataset('klue', 'mrc', split='train')
text_list = dataset[:100]['context']
documents = [Document(text=t) for t in text_list]

index = VectorStoreIndex.from_documents(documents)

print(dataset[0]['question'])

query_engine = index.as_query_engine(similarity_top_k=1, verbose=True)
response = query_engine.query(dataset[0]['question'])

print(response)

README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

wandb: Logged trace tree to W&B.


북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?


wandb: Logged trace tree to W&B.


한 달
